In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from keras import layers, models, optimizers, regularizers
from keras.applications import EfficientNetB0
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib.pyplot as plt

In [2]:
# test if running the GPU version of tensorflow
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
# loads directory of 3 datasets
dir_train = pd.read_csv('dataset/EuroSAT/train.csv')
dir_valid = pd.read_csv('dataset/EuroSAT/validation.csv')
dir_test = pd.read_csv('dataset/EuroSAT/test.csv')


In [5]:
img_dir = pd.concat([dir_train, dir_valid], ignore_index=False)
img_dir = img_dir.iloc[:,1:-1].reset_index().drop(['index'], axis=1)
print(img_dir.shape)
# img_dir = img_dir.iloc[:100, :] # limit sample size when testing
img_dir

(24300, 3)


,Filename,Label,ClassName
0,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,Industrial/Industrial_453.jpg,4,Industrial
4,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...
24295,SeaLake/SeaLake_1943.jpg,9,SeaLake
24296,AnnualCrop/AnnualCrop_211.jpg,0,AnnualCrop
24297,Industrial/Industrial_1428.jpg,4,Industrial
24298,AnnualCrop/AnnualCrop_2571.jpg,0,AnnualCrop


In [6]:
# Select a random, balanced subset of images
num_samples_per_class = 100
class_names = img_dir['ClassName'].unique()
subset_idx = []

for class_name in class_names:
    class_indices = img_dir[img_dir['ClassName'] == class_name].index
    random_indices = np.random.choice(class_indices, num_samples_per_class, replace=False)
    subset_idx.extend(random_indices)

subset_img_dir = img_dir.loc[subset_idx]

In [7]:
# Load images and labels/classes
images = []
classes = []
# labels.typeof()

base_path = 'dataset/EuroSAT/'
for index, row in subset_img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    classes.append(row['ClassName'])

# Normalize images
images = np.array(images) / 255.0
labels = pd.get_dummies(classes).values

In [8]:
images.shape

(1000, 224, 224, 3)

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Set up the data augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
valid_data_generator = ImageDataGenerator()

In [10]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"

# hub_layer = hub.KerasLayer(model_path, trainable=False)

def create_model(optimizer="adam", learning_rate=0.001, num_dense_layers=1, dropout_rate=0.5):
    optimizers = {
        "adam": Adam(learning_rate=learning_rate),
        "rmsprop": RMSprop(learning_rate=learning_rate)
    }
    
    hub_layer = hub.KerasLayer(MODEL_PATH, trainable=False)
    
    model = keras.Sequential()
    model.add(keras.layers.InputLayer((224, 224, 3)))
    model.add(hub_layer)
    
    for _ in range(num_dense_layers):
        model.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizers[optimizer], metrics=["accuracy"])
    
    return model

model = KerasClassifier(build_fn=create_model, epochs=3, batch_size=8, verbose=0)


C:\Users\Geran\AppData\Local\Temp\ipykernel_30236\1386802414.py:31: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=3, batch_size=8, verbose=0)


In [11]:
from sklearn.model_selection import KFold
import itertools, time

def custom_cross_val(param_grid, cv, X_train, y_train):
    all_param_combinations = list(itertools.product(*param_grid.values()))
    param_combinations_scores = []

    kfold = KFold(n_splits=cv, shuffle=True, random_state=42)

    for param_set in all_param_combinations:
        params = dict(zip(param_grid.keys(), param_set))
        print(f"Running cross validation with params: {params}")
        fold_scores = []

        for train_index, val_index in kfold.split(X_train, y_train):
            start_time = time.time()
            X_train_fold, y_train_fold = X_train[train_index], y_train[train_index]
            X_val_fold, y_val_fold = X_train[val_index], y_train[val_index]

            model = create_model(**params)
            model.fit(X_train_fold, y_train_fold)
            loss, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
            fold_scores.append(accuracy)
            print(f"Current Accuracy: {accuracy} || {time.time() - start_time} ")

        avg_accuracy = np.mean(fold_scores)
        print(f"Average accuracy: {avg_accuracy}")
        param_combinations_scores.append((params, avg_accuracy))

    return max(param_combinations_scores, key=lambda x: x[1])

param_grid = {
    "dropout_rate": [0.5, 0.2],
    "num_dense_layers": [1, 2],
    "learning_rate": [0.001, 0.0001],
}

# param_grid = {
#     "dropout_rate": [0.5, 0.2],
# }

best_params, best_accuracy = custom_cross_val(param_grid, cv=3, X_train=X_train, y_train=y_train)
print(f"Best params: {best_params} with accuracy: {best_accuracy}")

Running cross validation with params: {'dropout_rate': 0.5, 'num_dense_layers': 1, 'learning_rate': 0.001}
17/17 [==============================] - 29s 1s/step - loss: 1.9936 - accuracy: 0.5441
Current Accuracy: 0.7528089880943298 || 45.23429727554321 
17/17 [==============================] - 28s 1s/step - loss: 2.2429 - accuracy: 0.4803
Current Accuracy: 0.8089887499809265 || 43.72777962684631 
17/17 [==============================] - 29s 1s/step - loss: 1.8922 - accuracy: 0.5693
Current Accuracy: 0.7932330965995789 || 45.19144654273987 
Average accuracy: 0.7850102782249451
Running cross validation with params: {'dropout_rate': 0.5, 'num_dense_layers': 1, 'learning_rate': 0.0001}
17/17 [==============================] - 29s 1s/step - loss: 3.6870 - accuracy: 0.1632
Current Accuracy: 0.28838950395584106 || 45.092397928237915 
17/17 [==============================] - 30s 1s/step - loss: 3.6598 - accuracy: 0.1557
Current Accuracy: 0.2846441864967346 || 46.21269774436951 
17/17 [=========

ResourceExhaustedError: Graph execution error:

Detected at node 'activation/Gelu/truediv' defined at (most recent call last):
Node: 'activation/Gelu/truediv'
failed to allocate memory
	 [[{{node activation/Gelu/truediv}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_test_function_516465]